In [ ]:
# 1 commit

In [11]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# Глобальная переменная для хранения содержания лекции
lecture_content = ""

def read_lecture(file_path):
    global lecture_content
    with open(file_path, 'r', encoding='utf-8') as f:
        lecture_content = f.read()

def process_lecture():
    
    global lecture_content
    threshold=1.9
    cosine_similarity_threshold=0.8
    
    lecture_clean = re.sub(r'[^\w\s.\\n\n]', '', lecture_content)
    lecture_clean = lecture_clean.replace('\n', '')

    lecture_clean_words = lecture_clean.replace('.', ' ')
    
    stopWords = set(stopwords.words("russian"))
    words = word_tokenize(lecture_clean_words)

    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    sentences = sent_tokenize(lecture_clean)
    sentenceRole = dict()
    
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceRole:
                    sentenceRole[sentence] += freq
                else:
                    sentenceRole[sentence] = freq
    
    sumRole = sum(sentenceRole.values())
    averageRole = int(sumRole / len(sentenceRole))

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    summary_sentences = [sentences[i] for i in range(len(sentences))
                         if (sentenceRole[sentences[i]] > (threshold*averageRole) and
                             np.max(cosine_similarities[i, :]) > cosine_similarity_threshold)]

    summary = ' '.join(summary_sentences)
    
    return summary

# Список файлов для лекций
lecture_files = ["lecture1.txt", "lecture2.txt", "lecture3.txt", "lecture4.txt", "lecture5.txt"]

# Обработка каждой лекции отдельно
for i, lecture_file in enumerate(lecture_files):
    read_lecture(lecture_file)
    
    # Обработка лекции и вывод результатов
    lecture_summary = process_lecture()
    print(f"Summary for {lecture_file}:\n{lecture_summary}")


FileNotFoundError: [Errno 2] No such file or directory: 'lecture1.txt'